# Sparse Hebbian Learning with full homeostasis : testing different rescaling functions

In this notebook, we test the convergence of SparseNet as a function of different parameters tuning the quantization. These parameters only influence the way we select dictionary elements and thus the homeostasis.



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=2, suppress=True)

In [3]:
from shl_scripts.shl_experiments import SHL_set
tag = 'rescaling'
opts = dict(homeo_method='HEH', eta_homeo=0.05, C=10., nb_quant=256, P_cum=None, verbose=1)
opts = dict(homeo_method='HEH', n_iter=513, record_each=32, eta_homeo=0.1, C=5., nb_quant=256, P_cum=None, verbose=1)
opts = dict(homeo_method='HEH', verbose=1)

experiments = SHL_set(opts, tag=tag)
list_figures = []
list_figures = ['show_dico', 'show_Pcum']

Extracting data..loading the data called : data_cache/data_data
Data is of shape : (81920, 256) - done in 0.09s.


In [4]:
!ls -l {experiments.shl.data_cache}/{tag}*
!ls -l {experiments.shl.data_cache}/{tag}*lock*

-rw-r--r--  1 laurentperrinet  staff  2594598 Mar  7 13:46 data_cache/rescaling - C=1.00000_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2594598 Mar  7 17:58 data_cache/rescaling - C=1.58740_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2594598 Mar  7 22:49 data_cache/rescaling - C=10.07937_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2594598 Mar  7 23:37 data_cache/rescaling - C=16.00000_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2594598 Mar  7 20:12 data_cache/rescaling - C=2.51984_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2594598 Mar  7 21:08 data_cache/rescaling - C=4.00000_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2594598 Mar  7 21:59 data_cache/rescaling - C=6.34960_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  1628726 Mar  8 00:02 data_cache/rescaling - nb_quant=101_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  5534864 Mar  8 01:03 data_cache/rescaling - nb_quant=1024_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  1882646 Mar  8 00:14 data_cache/rescaling - 

## different rescaling values

In [ ]:
experiments.scan(variable='C', list_figures=[], display='')
plt.show()
fig, ax = experiments.scan(variable='C', list_figures=list_figures, display='final')
plt.show()

No cache found data_cache/rescaling - C=0.50000_dico.pkl: Learning the dictionary with algo = mp 
 Training on 81920 patches
[dict_learning] Iteration   1 /   4097 (elapsed time:   1s,   0mn   1s)
Iteration  129 /   4097 (elapsed time:  237s,   3mn  57s)
Iteration  257 /   4097 (elapsed time:  475s,   7mn  55s)
Iteration  385 /   4097 (elapsed time:  711s,  11mn  51s)
Iteration  513 /   4097 (elapsed time:  947s,  15mn  47s)
Iteration  641 /   4097 (elapsed time:  1183s,  19mn  43s)
Iteration  769 /   4097 (elapsed time:  1419s,  23mn  39s)
Iteration  897 /   4097 (elapsed time:  1665s,  27mn  45s)
Iteration  1025 /   4097 (elapsed time:  1902s,  31mn  42s)
Iteration  1153 /   4097 (elapsed time:  2137s,  35mn  37s)
Iteration  1281 /   4097 (elapsed time:  2374s,  39mn  34s)
Iteration  1409 /   4097 (elapsed time:  2880s,  48mn   0s)
Iteration  1537 /   4097 (elapsed time:  3392s,  56mn  32s)
Iteration  1665 /   4097 (elapsed time:  3905s,  65mn   5s)
Iteration  1793 /   4097 (elapsed 

## different quantization parameters

with a fixed $C$, we now change the number of quantization steps used to compute the histogram:

In [ ]:
experiments.scan(variable='nb_quant', list_figures=[], display='')
plt.show()
fig, ax = experiments.scan(variable='nb_quant', list_figures=list_figures, display='final')
plt.show()


## Version used

In [ ]:
%load_ext version_information
%version_information numpy, shl_scripts